In [1]:
from collections import defaultdict
import pandas as pd
from urllib.parse import unquote
import tqdm
import os
from bs4 import BeautifulSoup  
import numpy as np
import urllib
import re

In [2]:
path = 'raw/'

In [4]:
re.findall(r'/wiki/+\w*,?\b','/wiki/fsd_de_ded ddfdfsdsd sds ds sdsd /wiki/dfdf')

['/wiki/fsd_de_ded', '/wiki/dfdf']

In [5]:
topics = {}
for i, file in enumerate(os.listdir(path)):
    topics[file[:-5]] = i

In [6]:
len(topics)

2379

In [21]:
def get_matrix_re():
    matrix = np.zeros((len(os.listdir(path)), len(os.listdir(path))))
    df = defaultdict(lambda: [])
    for i in tqdm.tqdm_notebook(os.listdir(path)):
        with open (path+i, encoding="UTF-8") as file:
            read = unquote(file.read())
            cur_topic = i[:-5]
            link_site = re.findall(r'/wiki/.+?\b',read)
            for link in link_site:
                if link in topics.keys():
                    df[i.split('.')[0]].append(link.split('/')[-1])
                    matrix[topics[cur_topic]][topics[link[6:]]] = 1         
    return matrix

In [22]:
def get_matrix():
    matrix = np.zeros((len(os.listdir(path)), len(os.listdir(path))))
    for file in tqdm.tqdm_notebook(os.listdir(path)):
        with open(f"raw/{file}", encoding='utf-8') as f:
            cur_topic = file[:-5]
            bs = BeautifulSoup(f.read(), "html.parser")
            all_a = bs.find_all("a")
            for link in all_a:
                if 'href' in link.attrs:
                    href = urllib.parse.unquote(link['href'])
                    if href.startswith("/wiki/") and href[6:] in topics:
                        matrix[topics[cur_topic]][topics[href[6:]]] = 1
    return matrix

In [32]:
def process(matrix):
    for i in matrix:
        for j in range(len(i)):
            if i[j] == 0:
                i[j] = 0.001
    row_sums = matrix.sum(axis=1)
    norm_matrix = matrix / row_sums[:, np.newaxis]
    vector = np.array([0.2]*len(os.listdir(path)))
    for _ in range(100):
        vector = norm_matrix.transpose().dot(vector)
    # np.take(vector, [1, 2])
    rank = {}
    for i, value in enumerate(vector):
        rank[i] = value
    return rank

In [34]:
def page_rank(rank, n=10, word=None):
    if word is not None:
        new_topics = [ind for topic, ind in topics.items() if word.lower() in topic.lower()]
    if word is not None:
        new_rank = {}
        for ind in new_topics:
            new_rank[ind] = rank[ind]
    else:
        new_rank = rank.copy()
    items = sorted(new_rank.items(), key=lambda t: t[1])[::-1][:n]
    for index, value in items:
        print(f"{os.listdir(path)[index][:-5]}: {value}")

In [37]:
matrix = get_matrix()

In [38]:
rank = process(matrix)

In [39]:
page_rank(rank=rank)

Английский_язык: 6.710244106349573
США: 4.66682684199059
Франция: 3.891332065960388
Великобритания: 3.755379111713333
Германия: 3.0454025051563165
Соединённые_Штаты_Америки: 2.982485368742181
Италия: 2.41149748041042
СССР: 2.385908911034225
Викисклад: 2.3083234328050564
Латинский_язык: 2.3054123310637404


In [40]:
page_rank(rank=rank, word='армстронг')

Армстронг,_Луи: 0.21913073924573098
Армстронг,_Нил_Олден: 0.0820416765113164
Армстронг,_Нил: 0.07772736651558804
Армстронг,_Билли_Джо: 0.06033081219855844
Армстронг,_Алан: 0.05467384689837618
Армстронг_(фамилия): 0.054339929130478845
Армстронг,_Эдвин: 0.04872429424522622
Армстронг,_Лэнс: 0.04765582655907486
Билли_Джо_Армстронг: 0.04497463456555457
Армстронг,_Крэйг: 0.03901933675387288
